# Feynman Dataset

In [ ]:
import jax
import jax.numpy as jnp

from src.feynman import *
from src.utils import *

from jaxkan.KAN import KAN

from sklearn.model_selection import train_test_split

import optax
from flax import nnx

import os
import pandas as pd
import numpy as np

import warnings
from pandas.errors import ParserWarning
warnings.filterwarnings("ignore", category=ParserWarning)

# Create the directory if it doesn't exist
results_dir = "feynman_results"
os.makedirs(results_dir, exist_ok=True)

## Parameters

In [ ]:
# Setup
func_dict = {"f1": f1, "f2": f2, "f3": f3, "f4": f4, "f5": f5, "f6": f6, "f7": f7, "f8": f8, "f9": f9, "f10": f10,
             "f11": f11, "f12": f12, "f13": f13, "f14": f14, "f15": f15, "f16": f16, "f17": f17, "f18": f18, "f19": f19, "f20": f20}

func_dims = {"f1": 2, "f2": 3, "f3": 2, "f4": 2, "f5": 2, "f6": 2, "f7": 2, "f8": 2, "f9": 3, "f10": 2,
             "f11": 2, "f12": 2, "f13": 2, "f14": 3, "f15": 3, "f16": 2, "f17": 2, "f18": 3, "f19": 2, "f20": 3}

N = 5000
seed = 42

num_epochs = 2000

opt_type = optax.adam(learning_rate=0.001)

pow_basis = 1.75
pow_res = 0.25

# --------------------------
# Small architecture details
# --------------------------
G_small = 5
hidden_small = [8, 8]

params_small_baseline = {'k': 3, 'G': G_small, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                         'init_scheme': {'type': 'default'}}

params_small_glorot = {'k': 3, 'G': G_small, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                            'init_scheme': {'type': 'glorot', 'gain': None, 'distribution': 'uniform'}}

params_small_power = {'k': 3, 'G': G_small, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                      'init_scheme': {'type': 'power', "const_b": 1.0, "const_r": 1.0, "pow_b1": pow_basis, "pow_b2": pow_basis, "pow_r1": pow_res, "pow_r2": pow_res}}

# ------------------------
# Big architecture details
# ------------------------
G_big = 20
hidden_big = [32, 32, 32]

params_big_baseline = {'k': 3, 'G': G_big, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                         'init_scheme': {'type': 'default'}}

params_big_glorot = {'k': 3, 'G': G_big, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                            'init_scheme': {'type': 'glorot', 'gain': None, 'distribution': 'uniform'}}

params_big_power = {'k': 3, 'G': G_big, 'grid_range': (-1.0, 1.0), 'grid_e': 1.0, 'residual': nnx.silu, 'external_weights': True, 'add_bias': True,
                      'init_scheme': {'type': 'power', "const_b": 1.0, "const_r": 1.0, "pow_b1": pow_basis, "pow_b2": pow_basis, "pow_r1": pow_res, "pow_r2": pow_res}}

## Results

In [ ]:
small_experiment = "feynman_small"
small_file = os.path.join(results_dir, f"{small_experiment}.txt")

big_experiment = "feynman_big"
big_file = os.path.join(results_dir, f"{big_experiment}.txt")

# Define the headers
header = "function, method, run, loss, l2"

# Check if the file exists and write the header if it doesn't
if not os.path.exists(small_file):
    with open(small_file, "w") as file:
        file.write(header + "\n")

if not os.path.exists(big_file):
    with open(big_file, "w") as file:
        file.write(header + "\n")

In [ ]:
for func_name in func_dict.keys():
    print(f"Running Experiments for {func_name}.")
    function = func_dict[func_name]
    func_dim = func_dims[func_name]

    # Generate data
    x, y = generate_feyn_data(function, func_dim, N, seed)

    # Split data, only for uniformity with previous experiments, we don't use the test set anywhere
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)

    # Model input/output
    n_in, n_out = X_train.shape[1], y_train.shape[1]

    # Small architecture
    layer_dims = [n_in, *hidden_small, n_out]

    print(f"\tTraining model with dimensions {layer_dims}.")

    # For confidence
    for run in [1, 2, 3, 4, 5]:

        print(f"\t\tRun No. {run}.")

        # Baseline
        base_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_small_baseline, seed = seed+run)
        base_opt = nnx.Optimizer(base_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(base_model, base_opt, X_train, y_train)

        l2error = feyn_fit_eval(base_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, baseline, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(small_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tBaseline model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

        # Glorot
        glorot_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_small_glorot, seed = seed+run)
        glorot_opt = nnx.Optimizer(glorot_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(glorot_model, glorot_opt, X_train, y_train)

        l2error = feyn_fit_eval(glorot_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, glorot, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(small_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tGlorot model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

        # Power Law
        power_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_small_power, seed = seed+run)
        power_opt = nnx.Optimizer(power_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(power_model, power_opt, X_train, y_train)

        l2error = feyn_fit_eval(power_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, power, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(small_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tPower-law model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

    # Big architecture
    layer_dims = [n_in, *hidden_big, n_out]

    print(f"\tTraining model with dimensions {layer_dims}.")

    # For confidence
    for run in [1, 2, 3, 4, 5]:

        print(f"\t\tRun No. {run}.")

        # Baseline
        base_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_big_baseline, seed = seed+run)
        base_opt = nnx.Optimizer(base_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(base_model, base_opt, X_train, y_train)

        l2error = feyn_fit_eval(base_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, baseline, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(big_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tBaseline model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

        # Glorot
        glorot_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_big_glorot, seed = seed+run)
        glorot_opt = nnx.Optimizer(glorot_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(glorot_model, glorot_opt, X_train, y_train)

        l2error = feyn_fit_eval(glorot_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, glorot, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(big_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tGlorot model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

        # Power Law
        power_model = KAN(layer_dims = layer_dims, layer_type = 'spline', required_parameters = params_big_power, seed = seed+run)
        power_opt = nnx.Optimizer(power_model, opt_type)

        for epoch in range(num_epochs):
            loss = func_fit_step(power_model, power_opt, X_train, y_train)

        l2error = feyn_fit_eval(power_model, function, func_dim)

        # Log results
        new_row = f"{func_name}, power, {run}, {loss}, {l2error}"
                        
        # Append the row to the file
        with open(big_file, "a") as rfile:
            rfile.write(new_row + "\n")

        print(f"\t\t\tPower-law model. Final loss = {loss:.2e} \tRel. L2 Error: {l2error:.2e}")

## Analysis

In [ ]:
metric = "loss"   # l2 or loss
agg = "median"    # median or mean

sdf = pd.read_csv(small_file, sep=', ')
s_res = sdf.groupby(["function", "method"])[metric].agg(agg).reset_index()
s_table = s_res.pivot(index="function", columns="method", values=metric).reset_index()

bdf = pd.read_csv(big_file, sep=', ')
b_res = bdf.groupby(["function", "method"])[metric].agg(agg).reset_index()
b_table = b_res.pivot(index="function", columns="method", values=metric).reset_index()

In [ ]:
s_table

In [ ]:
b_table

In [ ]:
metric = "l2"   # l2 or loss
agg = "median"    # median or mean

sdf = pd.read_csv(small_file, sep=', ')
s_res = sdf.groupby(["function", "method"])[metric].agg(agg).reset_index()
s_table = s_res.pivot(index="function", columns="method", values=metric).reset_index()

bdf = pd.read_csv(big_file, sep=', ')
b_res = bdf.groupby(["function", "method"])[metric].agg(agg).reset_index()
b_table = b_res.pivot(index="function", columns="method", values=metric).reset_index()

In [ ]:
s_table

In [ ]:
b_table